In [3]:
import duckdb

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

In [4]:
df = pd.read_csv("poverty_raw_data.csv")
df.head(10)

,record_id,region_code,country_name,country_code,survey_acronym,reporting_year,reporting_level,survey_coverage,survey_year,welfare_type,...,decile9,decile10,cpi,ppp,reporting_pop,reporting_gdp,reporting_pce,is_interpolated,distribution_type,estimation_type
0,1,NaN,Angola,AGO,HBS,2000,national,national,2000.21,consumption,...,0.158687,0.402455,0.014200,107.269989,16394062,1840.353828,NaN,0,micro,survey
1,2,SSA,Angola,AGO,IBEP-MICS,2008,national,national,2008.50,consumption,...,0.162333,0.323117,0.303449,107.269989,21691522,3061.636272,1216.312411,0,micro,survey
2,3,SSA,Angola,AGO,IDREA,2018,national,national,2018.17,consumption,...,0.159390,0.396244,1.231024,107.269989,31273533,2677.249833,1548.153535,0,micro,survey
3,4,ECA,Armenia,ARM,ILCS,2012,national,national,2012.00,consumption,...,0.141960,0.245402,0.888864,167.312210,2914421,3278.670451,2953.453311,0,micro,survey
4,5,ECA,Albania,ALB,EWS,1996,national,national,1996.00,consumption,...,0.148995,0.212156,0.399635,50.357372,3168033,1633.552004,1716.336806,0,micro,survey
5,6,ECA,Albania,ALB,LSMS,2002,national,national,2002.00,consumption,...,0.149251,0.254356,0.701637,50.357372,3051010,2247.498086,1685.203083,0,micro,survey
6,7,ECA,Armenia,ARM,ILCS,2013,national,national,2013.00,consumption,...,0.147086,0.246855,0.940327,167.312210,2901385,3402.083858,3017.291522,0,micro,survey
7,8,NaN,Albania,ALB,LSMS,2005,national,national,2005.00,consumption,...,0.148339,0.243412,0.753958,50.357372,3011487,2675.508644,2079.203480,0,micro,survey
8,9,ECA,Armenia,ARM,ILCS,2014,national,national,2014.00,consumption,...,0.143123,0.257440,0.968361,167.312210,2889930,3538.529395,3077.428042,0,micro,survey
9,10,ECA,Albania,ALB,LSMS,2008,national,national,2008.00,consumption,...,0.145352,0.244583,0.820114,50.357372,2947314,3298.478917,2820.279521,0,micro,survey


In [6]:
query1 = duckdb.query('''SELECT
            country_name,
            survey_acronym,
            survey_year
            FROM df
            WHERE region_code ='LAC' and
            survey_coverage = 'national'
            GROUP BY 
            country_name,
            survey_acronym,
            survey_year;
            ''')
query1

┌─────────────────────┬────────────────┬─────────────┐
│    country_name     │ survey_acronym │ survey_year │
│       varchar       │    varchar     │   double    │
├─────────────────────┼────────────────┼─────────────┤
│ Bolivia             │ EH             │      2017.0 │
│ Brazil              │ PNAD           │      1983.0 │
│ Brazil              │ PNADC-E1       │      2016.0 │
│ Brazil              │ PNADC-E1       │      2017.0 │
│ Brazil              │ PNADC-E1       │      2018.0 │
│ Brazil              │ PNADC-E5       │      2021.0 │
│ Belize              │ LFS            │      1997.0 │
│ Brazil              │ PNAD           │      1998.0 │
│ Brazil              │ PNAD           │      1999.0 │
│ Brazil              │ PNAD           │      2007.0 │
│   ·                 │ ·              │         ·   │
│   ·                 │ ·              │         ·   │
│   ·                 │ ·              │         ·   │
│ Paraguay            │ EH             │      1995.0 │
│ El Salva

In [8]:
query1 = duckdb.query('''SELECT
            country_name,
            AVG(reporting_pce) AS min_avg_reporting_pce
            FROM df
            WHERE region_code ='SSA' 
            GROUP BY 
            country_name
            HAVING
            AVG(reporting_pce) <= ALL 
                (SELECT avg_reporting_pce FROM
                    (SELECT
                    country_name,
                    AVG(reporting_pce) AS avg_reporting_pce
                    FROM df
                    WHERE region_code ='SSA'
                    GROUP BY 
                    country_name)
                    WHERE avg_reporting_pce IS NOT NULL)''')
query1

┌──────────────────┬───────────────────────┐
│   country_name   │ min_avg_reporting_pce │
│     varchar      │        double         │
├──────────────────┼───────────────────────┤
│ Congo, Dem. Rep. │          305.26031975 │
└──────────────────┴───────────────────────┘

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2318 entries, 0 to 2317
Data columns (total 40 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   record_id             2318 non-null   int64  
 1   region_code           2216 non-null   object 
 2   country_name          2318 non-null   object 
 3   country_code          2318 non-null   object 
 4   survey_acronym        2318 non-null   object 
 5   reporting_year        2318 non-null   int64  
 6   reporting_level       2318 non-null   object 
 7   survey_coverage       2244 non-null   object 
 8   survey_year           2318 non-null   float64
 9   welfare_type          2318 non-null   object 
 10  survey_comparability  2318 non-null   int64  
 11  comparable_spell      2318 non-null   object 
 12  poverty_line          2318 non-null   float64
 13  headcount             2318 non-null   float64
 14  poverty_gap           2318 non-null   float64
 15  poverty_severity     

In [6]:
df[df["region_code"].isna()]

,record_id,region_code,country_name,country_code,survey_acronym,reporting_year,reporting_level,survey_coverage,survey_year,welfare_type,...,decile9,decile10,cpi,ppp,reporting_pop,reporting_gdp,reporting_pce,is_interpolated,distribution_type,estimation_type
0,1,NaN,Angola,AGO,HBS,2000,national,national,2000.21,consumption,...,0.158687,0.402455,0.014200,107.269989,16394062,1840.353828,NaN,0,micro,survey
7,8,NaN,Albania,ALB,LSMS,2005,national,national,2005.00,consumption,...,0.148339,0.243412,0.753958,50.357372,3011487,2675.508644,2079.203480,0,micro,survey
13,14,NaN,Albania,ALB,HBS,2016,national,national,2016.00,consumption,...,0.161388,0.250496,0.979810,50.357372,2876101,4090.372739,3262.756967,0,micro,survey
14,15,NaN,Albania,ALB,SILC-C,2016,national,national,2016.00,income,...,0.166304,0.274669,0.979810,50.357372,2876101,4090.372739,3262.756967,0,micro,survey
23,24,NaN,Armenia,ARM,ILCS,2018,national,national,2018.00,consumption,...,0.138003,0.291994,1.025202,167.312210,2836557,4215.899398,3392.411750,0,micro,survey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,681,NaN,Finland,FIN,IDS-LIS,1995,national,national,1995.00,income,...,0.139024,0.205439,4.323055,0.933301,5107790,29912.897970,15581.824660,0,micro,survey
685,686,NaN,Spain,ESP,EU-SILC,2012,national,national,2012.00,income,...,0.158622,0.257865,0.975502,0.702861,46773055,24634.962090,14704.794250,0,micro,survey
837,838,NaN,Gabon,GAB,EGEP,2005,national,national,2005.00,consumption,...,0.155274,0.331460,0.776824,319.160339,1458353,7155.365485,2150.297891,0,micro,survey
843,844,NaN,Gabon,GAB,EGEP,2017,national,national,2017.00,consumption,...,0.166351,0.277260,1.000000,319.160339,2140215,6893.384327,2612.306082,0,micro,survey


Approaches for Dealing with NULLs

Ignoring them

Deleting the rows containing them (listwise deletion)

Imputation based on theencie)scolumn

Imputation base on othe
(Imputation Based on Functional
Dependencie)r columns

In [7]:
region_codes = df[~df['region_code'].isna()].set_index("country_code")['region_code'].to_dict()

region_country_codes = df[df['region_code'].isna()]['country_code'].map(region_codes)
df['region_code'] = df['region_code'].fillna(region_country_codes)

In [8]:
df.loc[df['country_name'] == 'Gabon', 'region_code'] = df.loc[df['country_name'] == 'Gabon', 'region_code'].fillna('SSA')
df.loc[df['country_name'] == 'Guyana', 'region_code'] = df.loc[df['country_name'] == 'Guyana', 'region_code'].fillna('LAC')

In [9]:
df.groupby("survey_coverage")["record_id"].count()

survey_coverage
national    2186
rural          2
urban         56
Name: record_id, dtype: int64

Simple Univariate Imputation: Most frequent value observed in the column

In [10]:
df["survey_coverage"] = df["survey_coverage"].fillna("national")

In [11]:
df["reporting_pce"] = df.groupby("country_code")["reporting_pce"].transform(lambda x: x.fillna(x.mean()))

In [12]:
df["reporting_pce"] = df.groupby("region_code")["reporting_pce"].transform(lambda x: x.fillna(x.mean()))

In [13]:
duckdb.query('''SELECT
            country_name,
            survey_acronym,
            survey_year
            FROM df
            WHERE region_code ='LAC' and
            survey_coverage = 'national'
            GROUP BY 
            country_name,
            survey_acronym,
            survey_year
            ORDER BY country_name, survey_year
            ''')

┌───────────────┬────────────────┬─────────────┐
│ country_name  │ survey_acronym │ survey_year │
│    varchar    │    varchar     │   double    │
├───────────────┼────────────────┼─────────────┤
│ Argentina     │ EPHC-S2        │      2016.0 │
│ Belize        │ LFS            │      1993.0 │
│ Belize        │ LFS            │      1994.0 │
│ Belize        │ HBS            │      1995.0 │
│ Belize        │ SLC            │      1996.0 │
│ Belize        │ LFS            │      1997.0 │
│ Belize        │ LFS            │      1998.0 │
│ Belize        │ LFS            │      1999.0 │
│ Bolivia       │ ENE            │      1997.0 │
│ Bolivia       │ ECH            │      1999.0 │
│    ·          │  ·             │         ·   │
│    ·          │  ·             │         ·   │
│    ·          │  ·             │         ·   │
│ Venezuela, RB │ EHM            │      1992.0 │
│ Venezuela, RB │ EHM            │      1995.0 │
│ Venezuela, RB │ EHM            │      1998.0 │
│ Venezuela, RB │ EH

In [15]:
duckdb.query('''SELECT
            country_name,
            AVG(reporting_pce) AS min_avg_reporting_pce
            FROM df
            WHERE region_code ='SSA' 
            GROUP BY 
            country_name
            HAVING
            AVG(reporting_pce) <= ALL 
                (SELECT avg_reporting_pce FROM
                    (SELECT
                    country_name,
                    AVG(reporting_pce) AS avg_reporting_pce
                    FROM df
                    WHERE region_code ='SSA'
                    GROUP BY 
                    country_name)
                    WHERE avg_reporting_pce IS NOT NULL)''')

┌──────────────────┬───────────────────────┐
│   country_name   │ min_avg_reporting_pce │
│     varchar      │        double         │
├──────────────────┼───────────────────────┤
│ Congo, Dem. Rep. │          305.26031975 │
└──────────────────┴───────────────────────┘

1. country_code- country_name
2. country_code - region_code

In [16]:
countries = df[["country_code", "country_name"]].drop_duplicates()
regions = df[["country_code", "region_code"]].drop_duplicates()
df = df.drop(["country_name", "region_code"], axis=1)

In [17]:
duckdb.query('''SELECT
            C.country_name,
            survey_acronym,
            survey_year
            FROM df, regions AS R, countries AS C
            WHERE df.country_code = R.country_code AND
                df.country_code = C.country_code AND
                R.region_code ='LAC' AND
                survey_coverage = 'national'
            GROUP BY 
            country_name,
            survey_acronym,
            survey_year
            ORDER BY C.country_name, survey_year
            ''')

┌───────────────┬────────────────┬─────────────┐
│ country_name  │ survey_acronym │ survey_year │
│    varchar    │    varchar     │   double    │
├───────────────┼────────────────┼─────────────┤
│ Argentina     │ EPHC-S2        │      2016.0 │
│ Belize        │ LFS            │      1993.0 │
│ Belize        │ LFS            │      1994.0 │
│ Belize        │ HBS            │      1995.0 │
│ Belize        │ SLC            │      1996.0 │
│ Belize        │ LFS            │      1997.0 │
│ Belize        │ LFS            │      1998.0 │
│ Belize        │ LFS            │      1999.0 │
│ Bolivia       │ ENE            │      1997.0 │
│ Bolivia       │ ECH            │      1999.0 │
│    ·          │  ·             │         ·   │
│    ·          │  ·             │         ·   │
│    ·          │  ·             │         ·   │
│ Venezuela, RB │ EHM            │      1992.0 │
│ Venezuela, RB │ EHM            │      1995.0 │
│ Venezuela, RB │ EHM            │      1998.0 │
│ Venezuela, RB │ EH

In [25]:
duckdb.query('''SELECT
            C.country_name,
            AVG(reporting_pce) AS min_avg_reporting_pce
            FROM df, regions AS R, countries AS C
            WHERE df.country_code = R.country_code AND R.region_code ='SSA' AND
                df.country_code = C.country_code
            GROUP BY 
            C.country_name
            HAVING
            AVG(reporting_pce) <= ALL 
                (SELECT avg_reporting_pce FROM
                    (SELECT
                    df.country_code,
                    AVG(reporting_pce) AS avg_reporting_pce
                    FROM df, regions AS R
                    WHERE df.country_code = R.country_code AND R.region_code ='SSA'
                    GROUP BY 
                    df.country_code)
                    WHERE avg_reporting_pce IS NOT NULL)''')

┌──────────────────┬───────────────────────┐
│   country_name   │ min_avg_reporting_pce │
│     varchar      │        double         │
├──────────────────┼───────────────────────┤
│ Congo, Dem. Rep. │          305.26031975 │
└──────────────────┴───────────────────────┘